In [1]:
import tensorflow as tf
import numpy as np
import sys

from sklearn.metrics import mean_squared_error
from keras.callbacks import LearningRateScheduler

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression

#Data handlers
from ann_framework.data_handlers.data_handler_Oscillator import OscillatorDataHandler

#Custom modules
from ann_framework import aux_functions

import aux_functions_stochastic
import analytic_functions
import loss_functions

#global variables

deltas = [0.1, 0.1]

k = 1
c = 0.1
D = 1
num_fevals = 5

Using TensorFlow backend.


### Define tensorflow model

In [2]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y


def tf_model(X):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), 
                         name="fc1")
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="out")
    
    return y

### Create Model TF

In [3]:
def tf_compiled_model(num_features, output_shape):
    tf.reset_default_graph()

    X, y = create_placeholders(num_features, output_shape)

    y_pred = tf_model(X)
    loss_function = loss_functions.squared_residual_function_wrapper(k, c, D, deltas, num_fevals)
    cost = loss_function(X, y_pred)
    reg_cost = tf.losses.get_regularization_loss()
    total_cost = cost + reg_cost

    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.5).minimize(total_cost)

    return {'X_placeholder': X, 'y_placeholder': y, 'y_pred': y_pred, 'cost': cost, 'total_cost': total_cost, 'optimizer': optimizer}

### Create Tunable Model Tensorflow and assign data

In [4]:
dhandler_stochastic = OscillatorDataHandler()
model = tf_compiled_model(2, 1)

tModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='tensorflow', 
                                        data_handler=dhandler_stochastic)

tModel.load_data(verbose=1, cross_validation_ratio=0.2, x=[0, 0], boundaries=[5, 5], n=[50, 50])

tModel.print_data()

Loading data. Cros-Validation ratio 0.2
Printing shapes

Training data (X, y)
(7200, 2)
(7200, 1)
Cross-Validation data (X, y)
(1800, 2)
(1800, 1)
Testing data (X, y)
(1000, 2)
(1000, 1)
Printing first 5 elements

Training data (X, y)
[[ 2.07070707  0.45454545]
 [-1.86868687  3.98989899]
 [-3.18181818  4.09090909]
 [-4.39393939  0.55555556]
 [ 4.49494949  1.86868687]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Cross-Validation data (X, y)
[[ 1.66666667  2.37373737]
 [-2.27272727 -1.06060606]
 [-2.47474747 -0.35353535]
 [-4.6969697  -4.29292929]
 [-3.98989899 -1.86868687]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Testing data (X, y)
[[-1.56565657  4.19191919]
 [-2.67676768 -0.95959596]
 [ 3.58585859 -1.66666667]
 [-0.95959596  4.7979798 ]
 [ 1.06060606 -0.05050505]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Train model Tensorflow

In [5]:
tModel.epochs = 100
#lrate = LearningRateScheduler(aux_functions.step_decay)
minibatches_function_handle = aux_functions_stochastic.get_minibatches

#tf.get_variable("deltas", initializer=deltas)

sess = tf.Session()

#writer = tf.summary.FileWriter('./graphs', sess.graph)

tModel.train_model(tf_session=sess, get_minibatches_function_handle=minibatches_function_handle, 
                   verbose=1, deltas=deltas)

Epoch: 0001 cost_reg= 146.344486237 cost= 134.856307030
Epoch: 0002 cost_reg= 89.942028300 cost= 78.618686485
Epoch: 0003 cost_reg= 60.356043243 cost= 49.171475538
Epoch: 0004 cost_reg= 42.136371358 cost= 31.083258661
Epoch: 0005 cost_reg= 33.237255987 cost= 22.301865490
Epoch: 0006 cost_reg= 28.898372968 cost= 18.071075412
Epoch: 0007 cost_reg= 24.532900492 cost= 13.811147563
Epoch: 0008 cost_reg= 22.461543783 cost= 11.837905173
Epoch: 0009 cost_reg= 19.988274511 cost= 9.469326383
Epoch: 0010 cost_reg= 18.768446159 cost= 8.342166913
Epoch: 0011 cost_reg= 18.731622569 cost= 8.390961695
Epoch: 0012 cost_reg= 18.169512685 cost= 7.906285602
Epoch: 0013 cost_reg= 17.501372274 cost= 7.305515615
Epoch: 0014 cost_reg= 16.525266774 cost= 6.400338815
Epoch: 0015 cost_reg= 15.697854678 cost= 5.643863604
Epoch: 0016 cost_reg= 15.258254941 cost= 5.269584395
Epoch: 0017 cost_reg= 15.000322533 cost= 5.074573983
Epoch: 0018 cost_reg= 14.842544365 cost= 4.981878007
Epoch: 0019 cost_reg= 14.485422134 c

In [6]:
display_points = 10

#Evaluate real model
X_test = tModel.X_crossVal

#Real phi function
sigma_x = np.sqrt(D / (k * c))
sigma_y = np.sqrt(D / c)
p_real, phi_real = analytic_functions.real_p(X_test[:,0], X_test[:,1], sigma_x, sigma_y)

tModel.y_crossVal = phi_real

#p_real = np.ravel(p_real)
#phi_real = np.ravel(phi_real)

tModel.evaluate_model(['mse', 'rmse'], cross_validation=True, tf_session=sess)
#phi_pred = np.ravel(tModel.y_predicted)
phi_pred = tModel.y_predicted

d = 2 * np.pi * sigma_x * sigma_y
c_not = 1/d
p_pred = c_not * np.exp(-phi_pred)

print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
mse = cScores['mse']
time = tModel.train_time

total_points = len(phi_pred)
sample_array = list(range(total_points))

sample_points = np.random.choice(sample_array, display_points)

phi_real_sampled = phi_real[sample_points]
phi_pred_sampled = phi_pred[sample_points]

i = range(len(phi_pred_sampled))


for i, phi_real, phi_predicted in zip(i, phi_real, phi_pred):
    print('xy {}, Real Phi {}, Predicted Phi {}'.format(X_test[i], phi_real, phi_predicted))

#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("MSE: {}".format(mse))
print("Time : {} seconds".format(time))

scores
xy [1.66666667 2.37373737], Real Phi [0.42062034], Predicted Phi [0.02446432]
xy [-2.27272727 -1.06060606], Real Phi [0.31450872], Predicted Phi [0.02778697]
xy [-2.47474747 -0.35353535], Real Phi [0.31246812], Predicted Phi [0.02038467]
xy [-4.6969697  -4.29292929], Real Phi [2.02453831], Predicted Phi [0.02745827]
xy [-3.98989899 -1.86868687], Real Phi [0.97056423], Predicted Phi [0.03029504]
xy [-0.25252525  4.09090909], Real Phi [0.83996531], Predicted Phi [0.04042706]
xy [0.05050505 4.6969697 ], Real Phi [1.10320375], Predicted Phi [0.0449245]
xy [-3.28282828  0.55555556], Real Phi [0.55428018], Predicted Phi [0.04187646]
xy [-3.58585859  2.77777778], Real Phi [1.02872156], Predicted Phi [0.02849822]
xy [1.96969697 4.39393939], Real Phi [1.15932048], Predicted Phi [0.02996544]
RMSE2: 0.9742799003734285
MSE: 0.9492213242716578
Time : 3.3820069999999998 seconds
